In [34]:
import matplotlib.pyplot as plt
import numpy as np
#from sklearn.utils import shuffle # df=shuffle(df)
import pandas as pd
import tensorflow as tf

In [35]:
df=pd.read_csv('Iris.csv')
#print(df.head())

In [59]:


#sample(): returns a fraction of the original sample
#reset_index(drop=True):drop parameter to avoid the old index being added as a column:
df=df.sample(frac=1).reset_index(drop=True)#retain but shuffles data

print(df.head())

    Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  \
0    4            4.6           3.1            1.5           0.2   
1   59            6.6           2.9            4.6           1.3   
2   88            6.3           2.3            4.4           1.3   
3  145            6.7           3.3            5.7           2.5   
4  141            6.7           3.1            5.6           2.4   

           Species  
0      Iris-setosa  
1  Iris-versicolor  
2  Iris-versicolor  
3   Iris-virginica  
4   Iris-virginica  


In [60]:
def convert(str):
    if str == "Iris-setosa":
        return 0
    elif str == "Iris-virginica":
        return 2
    elif str == "Iris-versicolor":
        return 1
#indent

X=np.array(df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']])#shape (150,4)
Y=np.array(df['Species'].apply(convert))#get output shape(150,)

Z=np.eye(len(set(Y)))[Y]#one-hot encodings shape(150,3)
'''
ALGORITHM
1. get the input and output(done)
2. convert output to one-hot encoding(done)
3. do the train-test split(done)

4. find distance metric (done)
5. find k nearest neighbour
    5.1 find top k indices(done)
    5.2 find the associated top k values as one-hot encodings(done)
    5.3 add up the one-hot encodings for each value by axis=1(done)
'''



'\nALGORITHM\n1. get the input and output(done)\n2. convert output to one-hot encoding(done)\n3. do the train-test split(done)\n\n4. find distance metric (done)\n5. find k nearest neighbour\n    5.1 find top k indices(done)\n    5.2 find the associated top k values as one-hot encodings(done)\n    5.3 add up the one-hot encodings for each value by axis=1(done)\n'

In [68]:
np.random.seed(0)#non-deterministic seeding

#we are doing a 70:30 split
train_indice=np.random.choice(len(X), round(len(X)*0.7), replace=False)
test_indice=np.array(list(set(range(len(X))) - set(train_indice)))

#passed within feed_dict
Xtr=X[train_indice]
Ytr=Z[train_indice]
Xte=X[test_indice]
Yte=Z[test_indice]

In [69]:
# print(Xtr.shape)#features
# print(Xte.shape)#test features

# print(Ytr[0])#one-hot encoding
# print(Xte)

In [70]:
k = 5

x_data_train=tf.placeholder(shape=[None,len(Xtr[0])],dtype=tf.float32)#feed_dict{x_data_train:Xtr}
x_data_test=tf.placeholder(shape=[None,len(Xte[0])],dtype=tf.float32)#feed_dict{x_data_test:Xte}
y_data_train=tf.placeholder(shape=[None,len(Ytr[0])],dtype=tf.float32)#feed_dict{y_data_train:Ytr}

#MANHATTAN Distance
distance=tf.reduce_sum(tf.abs(x_data_train - tf.expand_dims(x_data_test, 1)), axis=2)


#transpose is expensive
#reshape is cheap

_,top_k_indice=tf.nn.top_k(tf.negative(distance),k=k)

#the following line generates a set of of one-hot encodings
#tf.gather(params,indices) accesses a set of values from "params" based on a set of "indices"
top_k_label=tf.gather(y_data_train,top_k_indice)


sum_up_predictions = tf.reduce_sum(top_k_label, axis=1)



In [71]:
prediction=tf.argmax(sum_up_predictions ,axis=1)

In [72]:
sess=tf.Session()
prediction_outcome=sess.run(prediction,feed_dict={x_data_train:Xtr,x_data_test:Xte,y_data_train:Ytr})

#print(sess.run(prediction,feed_dict={x_data_train:Xtr,x_data_test:Xte,y_data_train:Ytr}))


#ADDING AN EXTRA DIMENSION
# dummy=tf.expand_dims(x_data_test, 0)#[?,4] to shape = (1,?,4)
# #tf.expand_dims(x_data_test, 1)#[?,4] to (?,1,4)
# #tf.expand_dims(x_data_test, 2)#[?,4] to (?,4,1)
# print(dummy.shape)
# print(sess.run(dummy,feed_dict={x_data_test:Xte}))


# tf.shape(top_k_label) #[45,5,3]
# [[ 0.  1.  0.]
#   [ 0.  1.  0.]
#   [ 0.  1.  0.]
#   [ 0.  0.  1.]
#   [ 0.  0.  1.]]

# tf.shape(sum_up_predictions) #[45,3]
# [ 0.  3.  2.]

# tf.shape(prediction) #[45]
# 1


In [73]:
#FINDING THE ACCURACY
accuracy=0
for pred,actual in zip(prediction_outcome,Yte):
    if pred == np.argmax(actual):
        accuracy+= 1
        
print(accuracy/len(prediction_outcome))


0.9777777777777777
